In [1]:
import torch
import pickle

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from ds.drivers.datasets import DriversDataset, all_data

from torchsummary import summary

In [2]:
seed = 1337
np.random.seed(seed)
torch.random.manual_seed(seed)

In [3]:
ds_all_train = DriversDataset(
    all_data, ds_type="train", data_field="working_data_filtered", step=5
)
ds_all_test = DriversDataset(
    all_data, ds_type="test", data_field="working_data_filtered", step=5
)

In [4]:
base_path = "/home/dmo/Documents/human_func_state"
epoch_count = 1000
optim_lr = 7.5e-5
betas = "default"  # (0.85, 0.995) # Adam only
train_batch_size = 8
test_batch_size = 1
device = "cuda"

In [4]:
dl_train = DataLoader(
    ds_all_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

NameError: name 'train_batch_size' is not defined

In [6]:
dl_test = DataLoader(
    ds_all_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [7]:
class ConvX(nn.Module):
    def __init__(
        self, in_planes, out_planes, kernel=3, stride=1, padding=None
    ):
        super(ConvX, self).__init__()
        padding = kernel // 2 if padding is None else padding
        self.conv = nn.Conv1d(
            in_planes,
            out_planes,
            kernel_size=kernel,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [8]:
class NetUpDownCoder3(nn.Module):
    def __init__(self):
        super(NetUpDownCoder3, self).__init__()
        self.seq = torch.nn.Sequential(
            ConvX(1, 4, kernel=3),
            ConvX(4, 8, kernel=3),
            nn.MaxPool1d(2),
            ConvX(8, 16, kernel=3),
            ConvX(16, 32, kernel=3),
            nn.MaxPool1d(2),
            ConvX(32, 16, kernel=3),
            ConvX(16, 8, kernel=3),
            nn.MaxPool1d(2),
            ConvX(8, 4, kernel=3),
            ConvX(4, 2, kernel=3),
            nn.Conv1d(2, 2, kernel_size=3, stride=1),
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.seq(x)
        x = torch.flatten(x, 1)
        return self.softmax(x)

In [9]:
net = NetUpDownCoder3().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.ASGD(net.parameters())

In [10]:
mod_name = (
    f"{net.__class__.__name__}"
    f"_{optimizer.__class__.__name__}"
    f"_lr_{optim_lr}"
)
write_path = f"{base_path}/models_dumps/drivers/{mod_name}"
writer = SummaryWriter(log_dir=f"{write_path}/hist")
write_path = f"{write_path}/{mod_name}"

In [11]:
def val(model):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl_test):
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            outputs = model(inputs).cpu()
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [12]:
def train(
    model,
    epoch_count=50,
    print_step=50,
    start_epoch=0,
    min_loss=(np.inf, 0),
    best_rate=(0.0, 0),
    worst_rate=(1.0, 0),
):
    len_test = len(dl_test) * dl_test.batch_size
    for epoch in range(
        start_epoch, epoch_count
    ):  # loop over the dataset multiple times
        epoch_loss = 0
        running_loss = 0.0
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.cpu(), labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step - 1:
                mean_loss = running_loss / print_step
                if mean_loss < min_loss[0]:
                    min_loss = (mean_loss, (epoch, i))
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {mean_loss:.3f}")
                epoch_loss += running_loss
                running_loss = 0.0
        epoch_loss += running_loss
        model.eval()
        count_of_correct = val(model)
        model.train()
        rate = count_of_correct / len_test
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", rate, epoch)
        if rate > best_rate[0]:
            best_rate = (rate, epoch)
            with open(f"{write_path}.pkl", "wb") as f:
                state_dict = net.state_dict()
                pickle.dump(
                    {
                        "net_state_dict": state_dict,
                        "current_epoch": epoch,
                        "rate": rate,
                        "optimizer": optimizer.__class__.__name__,
                        "optimizer_params": optimizer.param_groups,
                        "train_batch_size": train_batch_size,
                        "test_batch_size": test_batch_size,
                        "device": device,
                    },
                    f,
                )
        if rate < worst_rate[0]:
            worst_rate = (rate, epoch)

        print(
            f"[{epoch + 1:3}] rate: {rate:.4f} - {count_of_correct:3d}/{len_test}; {best_rate = }, {worst_rate = }"
        )
    print("Finished Training. Min_loss:", min_loss)
    return worst_rate, best_rate, min_loss

In [13]:
train(net, epoch_count=epoch_count)

[1,    50] loss: 0.700
[1,   100] loss: 0.679
[1,   150] loss: 0.652
[1,   200] loss: 0.659
[1,   250] loss: 0.649
[1,   300] loss: 0.646
[1,   350] loss: 0.650
[1,   400] loss: 0.641
[1,   450] loss: 0.611
[  1] rate: 0.3556 - 336/945; best_rate = (tensor(0.3556), 0), worst_rate = (tensor(0.3556), 0)
[2,    50] loss: 0.627
[2,   100] loss: 0.631
[2,   150] loss: 0.601
[2,   200] loss: 0.621
[2,   250] loss: 0.620
[2,   300] loss: 0.623
[2,   350] loss: 0.617
[2,   400] loss: 0.600
[2,   450] loss: 0.609
[  2] rate: 0.3566 - 337/945; best_rate = (tensor(0.3566), 1), worst_rate = (tensor(0.3556), 0)
[3,    50] loss: 0.590
[3,   100] loss: 0.627
[3,   150] loss: 0.620
[3,   200] loss: 0.633
[3,   250] loss: 0.620
[3,   300] loss: 0.589
[3,   350] loss: 0.593
[3,   400] loss: 0.572
[3,   450] loss: 0.604
[  3] rate: 0.6550 - 619/945; best_rate = (tensor(0.6550), 2), worst_rate = (tensor(0.3556), 0)
[4,    50] loss: 0.567
[4,   100] loss: 0.590
[4,   150] loss: 0.574
[4,   200] loss: 0.574

((tensor(0.3323), 7), (tensor(0.7090), 4), (0.31830491840839387, (764, 149)))